In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
# https://www.kaggle.com/datasets/sriharshaeedala/financial-fraud-detection-dataset/data
!kaggle datasets download -d sriharshaeedala/financial-fraud-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/sriharshaeedala/financial-fraud-detection-dataset
License(s): CC-BY-SA-4.0
 95% 169M/178M [00:06<00:00, 48.5MB/s]
100% 178M/178M [00:06<00:00, 28.5MB/s]


In [3]:
!unzip financial-fraud-detection-dataset.zip

Archive:  financial-fraud-detection-dataset.zip
  inflating: Synthetic_Financial_datasets_log.csv  


In [4]:
! pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.6/565.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [5]:
!pip install -q pyngrok

In [6]:
!ngrok config add-authtoken 2nJFbyBQCio0ghSzxaqk1S44Zmj_osP2efhkNEJT69J3FPxq

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:

from pyngrok import ngrok

# Run the MLflow UI and expose it via ngrok
get_ipython().system_raw("mlflow ui --port 5000 &")
ngrok_tunnel = ngrok.connect(5000)
print("MLflow UI:", ngrok_tunnel.public_url)

MLflow UI: https://faae-34-16-151-250.ngrok-free.app


In [8]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [9]:
# Load the dataset
df = pd.read_csv("/content/Synthetic_Financial_datasets_log.csv")

print(f"Number of samples: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")
df.head()

Number of samples: 6362620
Number of features: 11


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [10]:
# selecting the columns of numerical type
numeric_columns = df.select_dtypes(include=['int', 'float']).columns

numeric_data = df[numeric_columns]

# pearson corrleation matrix of the numerical data
correlation = numeric_data.corr()

In [11]:
print(f"correlation between all features and Class \n{(correlation['isFraud'].sort_values(ascending=False))}")

correlation between all features and Class 
isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


In [12]:
target_correlation = correlation['isFraud'].drop('isFraud')

In [13]:
threshold= 0.02
features=target_correlation[abs(target_correlation) >= threshold].index

In [14]:
features=features[:2]

In [15]:
copy_df=df[features].join(df['type'])
copy_df=copy_df.join(df['isFraud'])
df=copy_df

In [16]:
df.head()

,step,amount,type,isFraud
0,1,9839.64,PAYMENT,0
1,1,1864.28,PAYMENT,0
2,1,181.00,TRANSFER,1
3,1,181.00,CASH_OUT,1
4,1,11668.14,PAYMENT,0


In [17]:
# Encode the 'type' column
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])

In [18]:
df = df.dropna(subset=['isFraud'])

In [19]:
# Separate feature variables and target variable
X = df.drop('isFraud', axis=1)
y = df['isFraud']

In [20]:
# Standardize the data
scaler = MinMaxScaler()
df['amount'] = scaler.fit_transform(df[['amount']])

In [21]:
# Initialize SMOTE and apply it to the training data only
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [22]:
df2 = pd.DataFrame(X_smote)
df2['isFraud'] = y_smote
df2

,step,amount,type,isFraud
0,1,9.839640e+03,3,0
1,1,1.864280e+03,3,0
2,1,1.810000e+02,4,1
3,1,1.810000e+02,1,1
4,1,1.166814e+04,3,0
...,...,...,...,...
12708809,253,1.620903e+06,1,1
12708810,617,1.000000e+07,1,1
12708811,110,9.041246e+05,1,1
12708812,48,1.367124e+05,2,1


In [23]:
df2.shape

(12708814, 4)

In [24]:
df2.drop_duplicates(inplace=True)
df2.shape

(11260389, 4)

In [25]:
# Separate feature variables and target variable
X = df2.drop('isFraud', axis=1)
y = df2['isFraud']

In [26]:
# Split the data into train (80%), validation (10%), and test sets (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [27]:
mlflow.set_experiment("Fraud_Detection_Experiment")

2024/10/16 12:07:01 INFO mlflow.tracking.fluent: Experiment with name 'Fraud_Detection_Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/157147915401937966', creation_time=1729080421135, experiment_id='157147915401937966', last_update_time=1729080421135, lifecycle_stage='active', name='Fraud_Detection_Experiment', tags={}>

In [28]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [29]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score # Import accuracy_score here
from sklearn.metrics import classification_report # you will likely need classification_report too
DT=DecisionTreeClassifier()
# Start a new run
with mlflow.start_run():
    DT.fit(X_train, y_train)
    mlflow.sklearn.log_model(DT, "Decision_Tree_model")

# Validate the model on the validation set
y_val_pred = DT.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred)}")
print(f"Validation Classification Report:\n {classification_report(y_val, y_val_pred)}")
mlflow.log_metric("Validation Accuracy", accuracy_score(y_val, y_val_pred))

# Test the model on the test set
y_test_pred = DT.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Test Classification Report:\n {classification_report(y_test, y_test_pred)}")
mlflow.log_metric("Test Accuracy", accuracy_score(y_test, y_test_pred))

2024/10/16 12:08:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Validation Accuracy: 0.975753060062751
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98    634692
           1       0.96      0.98      0.97    491347

    accuracy                           0.98   1126039
   macro avg       0.97      0.98      0.98   1126039
weighted avg       0.98      0.98      0.98   1126039

Test Accuracy: 0.975833874315188
Test Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98    634692
           1       0.96      0.98      0.97    491347

    accuracy                           0.98   1126039
   macro avg       0.97      0.98      0.98   1126039
weighted avg       0.98      0.98      0.98   1126039



In [31]:
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [34]:
mlflow.set_tag("mlflow.runName", "Decision_Tree_model")
# Train the Decision Tree model
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)
mlflow.sklearn.log_model(model, "Decision_Tree_model")
# Make predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
mlflow.log_metric("AUC", roc_auc_score(y_test, y_proba))
# Evaluate metrics
auc_score = roc_auc_score(y_test, y_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Log model parameters and metrics in MLflow
mlflow.log_param("max_depth", 5)
mlflow.log_metric("auc_score", auc_score)
mlflow.log_metric("precision", precision)
mlflow.log_metric("recall", recall)
mlflow.log_metric("f1_score", f1)


# Print the results
print(f"AUC: {auc_score}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")
mlflow.log_metric("AUC", auc_score)
mlflow.log_metric("Precision", precision)
mlflow.log_metric("Recall", recall)
mlflow.log_metric("F1 Score", f1)

2024/10/16 12:11:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.908771033354076, Precision: 0.8710431370643472, Recall: 0.7296615223050105, F1 Score: 0.7941085916924897


In [35]:
print("Navigate to the MLflow UI to see the experiment details:", ngrok_tunnel.public_url)

Navigate to the MLflow UI to see the experiment details: https://faae-34-16-151-250.ngrok-free.app
